In [0]:
from pyspark.sql.utils import AnalysisException

start_task = "job_logging_start"
all_tasks = [
    "FetchingPropertyAPI",
    "Bronze_Layer_Property",
    "check_for_property_data",
    "silver_layer_property",
    "move_to_archive"
]
debug_job_id = "unknown_job_id"

# STEP 1: Get job_id from start task
try:
    job_id = dbutils.jobs.taskValues.get(
        taskKey=start_task,
        key="job_id",
        debugValue=debug_job_id
    )
except Exception:
    job_id = debug_job_id

# STEP 2: Evaluate job status
job_status = "Succeeded"
skipped_detected = False

for task in all_tasks:
    try:
        error = dbutils.jobs.taskValues.get(taskKey=task, key="error", debugValue=None)
        if error:  # Explicit failure
            job_status = "Failed"
            break
    except Exception:
        skipped_detected = True  # Could not access task's status, likely skipped

# Finalize status
if job_status != "Failed":
    job_status = "Skipped" if skipped_detected else "Succeeded"

# STEP 3: Update job_control table
update_query = f"""
    UPDATE zillow.propertyextended.job_control
    SET
        job_status = '{job_status}',
        end_time = current_timestamp()
    WHERE
        job_id = '{job_id}'
"""

try:
    spark.sql(update_query)
    print(f"Job status updated for job_id = {job_id} with status = {job_status}")
except AnalysisException as sql_ex:
    print(f"SQL error while updating job_control: {sql_ex}")
except Exception as ex:
    print(f"Unexpected error while updating job_control: {ex}")


In [0]:
%sql
select * from zillow.propertyextended.job_control